In [299]:
import pprint

class PrintFuncIO:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print('[input] tuple params : ', end='')
        pprint.pprint(args)
        print('[input]dict params : ', end='')
        pprint.pprint(kwargs)
        print('[output]result : ', end='')
        result = self.func(*args, **kwargs)
        pprint.pprint(result)
        return result

@PrintFuncIO
def add(a, b):
    return a + b

add(20, 30)

[input] tuple params : (20, 30)
[input]dict params : {}
[output]result : 50


50

<hr>

<hr>

<hr>

<hr>

<hr>

In [289]:
import json
import pprint

class IndyPrinter:
    
    line_char='-'
    line_length=100
    ref_line_char='#'
    ref_line_length=50
    
    class __JsonPrintType:
        NOT_JSON = 0
        PURE_JSON = 1
        MULTIPLE_JSON_STR = 2
        
    def __init__(self, data):
        self.__result, self.__is_json = self.__type_filter(data)
        
    def __call__(self, print_title=None, *, use_pp=False):
        pp = pprint.PrettyPrinter(width=40, indent=2, compact=True)
        
        self.__line(self.line_char, self.line_length)
        if print_title is not None : print(print_title.center(self.line_length, self.line_char))
        if not self.__is_json : print(self.__result)
        elif use_pp : pp.pprint(self.__result)
        else : self.__json_print()
        self.__line(self.line_char, self.line_length)
    
    def default_mode(self):
        self.line_char='-'
        self.line_length=100
        self.ref_line_char='^'
        self.ref_line_length=50
        
    def __type_filter(self, data):
        _data = data
        data_type = type(_data)
        is_json = True
        
        try:
            if data_type == 'dict' : 
                _data = json.dumps(_data)
            _data = json.loads(_data)
        except json.JSONDecodeError:
            is_json = False
        finally:
            if(is_json) : return (_data, is_json)
            return (_data, is_json)
    
    def __line(self, char, length):
        print(char * length)
    
    def __json_print(self):
        json_obj = self.__result
        print(f'<json_keys> : {[key for key in json_obj]}')
        spec = self.__analyze_structure(json_obj)
        json_info, references = self.__mapping(json_obj, spec)
        print(json.dumps(json_info, indent=4))
        self.__line(self.ref_line_char, self.ref_line_length)
        print("REFERENCES".center(self.ref_line_length, self.ref_line_char))
        for ref_name in references: 
            print(f'{ref_name} : {json.dumps(references[ref_name], indent=4)}')
        self.__line(self.ref_line_char, self.ref_line_length)
    
    def __analyze_structure(self, json_obj):
        result = []
        for k, v in data.items():
            parse_type = None
            if type(v) != int and v[0] == '{' and v[-1] == '}' : 
                parse_type = self.__JsonPrintType.PURE_JSON
                if '\n' in v : 
                    parse_type = self.__JsonPrintType.MULTIPLE_JSON_STR
            else: 
                parse_type = self.__JsonPrintType.NOT_JSON
            result.append((k, parse_type))
        return dict(result)
    
    def __mapping(self, json_obj, spec):
        
        details = dict()
        
        for key in json_obj:
            if spec[key] == self.__JsonPrintType.MULTIPLE_JSON_STR:
                name = '[[{}]]'.format(key)
                details[name] = list(map(lambda json_str: json.loads(json_str), json_obj[key].split('\n')))
                json_obj[key] = f'{name} : please ignore list'
            elif spec[key] == self.__JsonPrintType.PURE_JSON:
                json_obj[key] = json.loads(json_obj[key])
            elif spec[key] == self.__JsonPrintType.NOT_JSON:
                pass
            
        return json_obj, details

In [288]:
import pprint

data = '{"result":{"txn":{"type":"113","data":{"revocDefType":"CL_ACCUM","id":"DYHgp6eSHgiuzJEekw44c2:4:DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1:CL_ACCUM:TAG1","tag":"TAG1","credDefId":"DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1","value":{"issuanceType":"ISSUANCE_ON_DEMAND","maxCredNum":5,"tailsHash":"F85KiXLLyyn2uh5hoiDFseLZR31QTQg6PkocELfwuR4L","tailsLocation":"\\/tmp\\/indy_acme_tails\\/F85KiXLLyyn2uh5hoiDFseLZR31QTQg6PkocELfwuR4L","publicKeys":{"accumKey":{"z":"1 13373A9FCA6E8E8A12751EE52C184B57C72F76E3AF7E5F6D1920A5327298FE71 1 09EF96C87BF295B578023CC69C5CF9CC7D52E33D1E72F09EC396BE330C1A9A93 1 130DC876C1CD310DF1ABEF61882004A3033A1EACA447C7D94A36E37D816F79B6 1 1FD463913FD441A327102CF6808B2B53E495E3B1599EBE901972184E00D0B7DD 1 0104B475C176D685271A6562E6F3B7BBF88C676C2BE66526F2FA8FFBB3B6F4C3 1 005FC6A72BC2A2EC9D44E4AE4D1205F7AA1BC81D12081B67B5C5902B7FB1952F 1 109E91E52D014EFAC2B7ADFB24F98EDB9E0D21376A9D6C045131883F846AD2AE 1 2004F11E3CCDA5193FF49737BB7FB96C928868EF1DA4B704BBFC7E7302EF93C9 1 2273A3E6A7ED9343C64809E44DF9D7CD660AEAE98A9BCCE53628B150A659BC5C 1 1113CEBBD0B4091A8823B9F873EBBE42F9236954950C3D532E72A0CEA0044FB9 1 1EDD169DC170D699D4585BFCF89FBC846DCD948446383B3633DE4BCA26077B61 1 06B7B2041D2F2B35D5C57722A9BE0207EFFA9B40C7AA36C7D2CC929E2DBDA185"}}}},"metadata":{"payloadDigest":"c769fd40b3437e59f38e3a4230adff41519e9d88b9e838983a5a0fd9ed40c646","from":"DYHgp6eSHgiuzJEekw44c2","reqId":1627591155738079055,"digest":"de52c5eb78042217fa19b878a2660f9566412ee656d0db590938b65bac9be45e"},"protocolVersion":2},"txnMetadata":{"txnId":"DYHgp6eSHgiuzJEekw44c2:4:DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1:CL_ACCUM:TAG1","txnTime":1627591176,"seqNo":110},"auditPath":["43d9XatbRrfT6eH1NGfrZWvFiDEirfde2vQbjV26ycXA","6r3vYcCLa8Cpvn8AEKifg464UwyE8JPeHoUMZxZEGGMS","yfYMJ83bPsKVHDVMjxupgNWkL9CArAUpLQ3oN9zkNTh","6dvhke25RyaL7KEDjo6Qz8d8gmwRbmJ3akurLQJrWYyr","ChXSt8Tk59pd6re6jEL1LiDxHRrzQ2BFnpms9dMCANFW"],"rootHash":"6nYk14b3aH8WZjB1F7nnAGpi42tQTDnoWbSfCRgHfP62","ver":"1","reqSignature":{"type":"ED25519","values":[{"from":"DYHgp6eSHgiuzJEekw44c2","value":"26qy1xGFkFDgWBc3KpN4S9V8aLoTkZnqgi3M9DWgNpD555KnkVYhqqYRZSyxC5JfLHq19YbaDYm8AVGvMZAtiUEF"}]}},"op":"REPLY"}'

IndyPrinter(data)('hello', use_pp=True)

----------------------------------------------------------------------------------------------------
-----------------------------------------------hello------------------------------------------------
{ 'op': 'REPLY',
  'result': { 'auditPath': [ '43d9XatbRrfT6eH1NGfrZWvFiDEirfde2vQbjV26ycXA',
                             '6r3vYcCLa8Cpvn8AEKifg464UwyE8JPeHoUMZxZEGGMS',
                             'yfYMJ83bPsKVHDVMjxupgNWkL9CArAUpLQ3oN9zkNTh',
                             '6dvhke25RyaL7KEDjo6Qz8d8gmwRbmJ3akurLQJrWYyr',
                             'ChXSt8Tk59pd6re6jEL1LiDxHRrzQ2BFnpms9dMCANFW'],
              'reqSignature': { 'type': 'ED25519',
                                'values': [ { 'from': 'DYHgp6eSHgiuzJEekw44c2',
                                              'value': '26qy1xGFkFDgWBc3KpN4S9V8aLoTkZnqgi3M9DWgNpD555KnkVYhqqYRZSyxC5JfLHq19YbaDYm8AVGvMZAtiUEF'}]},
              'rootHash': '6nYk14b3aH8WZjB1F7nnAGpi42tQTDnoWbSfCRgHfP62',
              'txn': { 'data': { 'credDef

In [276]:
data = {'name': 'pool1', 'genesis_txn_path': '/tmp/indy/pool1.txn', 'config': '{"genesis_txn": "/tmp/indy/pool1.txn"}', 'genesis_txn_data': '{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"127.0.0.1","client_port":9702,"node_ip":"127.0.0.1","node_port":9701,"services":["VALIDATOR"]},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"},"metadata":{"from":"Th7MpTaRZVRYnPiabds81Y"},"type":"0"},"txnMetadata":{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"},"ver":"1"}\n{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG7zqS8TK1MXwuCHj1FKNzVpsnafmqLG1vXN88rt38mNFs9TENzm4QHdBzsvCuoBnPH7rpYYDo9DZNJePaDvRvqJKByCabubJz3XXKbEeshzpz4Ma5QYpJqjk","blskey_pop":"Qr658mWZ2YC8JXGXwMDQTzuZCWF7NK9EwxphGmcBvCh6ybUuLxbG65nsX4JvD4SPNtkJ2w9ug1yLTj6fgmuDg41TgECXjLCij3RMsV8CwewBVgVN67wsA45DFWvqvLtu4rjNnE9JbdFTc1Z4WCPA3Xan44K1HoHAq9EVeaRYs8zoF5","client_ip":"127.0.0.1","client_port":9704,"node_ip":"127.0.0.1","node_port":9703,"services":["VALIDATOR"]},"dest":"8ECVSk179mjsjKRLWiQtssMLgp6EPhWXtaYyStWPSGAb"},"metadata":{"from":"EbP4aYNeTHL6q385GuVpRV"},"type":"0"},"txnMetadata":{"seqNo":2,"txnId":"1ac8aece2a18ced660fef8694b61aac3af08ba875ce3026a160acbc3a3af35fc"},"ver":"1"}\n{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node3","blskey":"3WFpdbg7C5cnLYZwFZevJqhubkFALBfCBBok15GdrKMUhUjGsk3jV6QKj6MZgEubF7oqCafxNdkm7eswgA4sdKTRc82tLGzZBd6vNqU8dupzup6uYUf32KTHTPQbuUM8Yk4QFXjEf2Usu2TJcNkdgpyeUSX42u5LqdDDpNSWUK5deC5","blskey_pop":"QwDeb2CkNSx6r8QC8vGQK3GRv7Yndn84TGNijX8YXHPiagXajyfTjoR87rXUu4G4QLk2cF8NNyqWiYMus1623dELWwx57rLCFqGh7N4ZRbGDRP4fnVcaKg1BcUxQ866Ven4gw8y4N56S5HzxXNBZtLYmhGHvDtk6PFkFwCvxYrNYjh","client_ip":"127.0.0.1","client_port":9706,"node_ip":"127.0.0.1","node_port":9705,"services":["VALIDATOR"]},"dest":"DKVxG2fXXTU8yT5N7hGEbXB3dfdAnYv1JczDUHpmDxya"},"metadata":{"from":"4cU41vWW82ArfxJxHkzXPG"},"type":"0"},"txnMetadata":{"seqNo":3,"txnId":"7e9f355dffa78ed24668f0e0e369fd8c224076571c51e2ea8be5f26479edebe4"},"ver":"1"}\n{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node4","blskey":"2zN3bHM1m4rLz54MJHYSwvqzPchYp8jkHswveCLAEJVcX6Mm1wHQD1SkPYMzUDTZvWvhuE6VNAkK3KxVeEmsanSmvjVkReDeBEMxeDaayjcZjFGPydyey1qxBHmTvAnBKoPydvuTAqx5f7YNNRAdeLmUi99gERUU7TD8KfAa6MpQ9bw","blskey_pop":"RPLagxaR5xdimFzwmzYnz4ZhWtYQEj8iR5ZU53T2gitPCyCHQneUn2Huc4oeLd2B2HzkGnjAff4hWTJT6C7qHYB1Mv2wU5iHHGFWkhnTX9WsEAbunJCV2qcaXScKj4tTfvdDKfLiVuU2av6hbsMztirRze7LvYBkRHV3tGwyCptsrP","client_ip":"127.0.0.1","client_port":9708,"node_ip":"127.0.0.1","node_port":9707,"services":["VALIDATOR"]},"dest":"4PS3EDQ3dW1tci1Bp6543CfuuebjFrg36kLAUcskGfaA"},"metadata":{"from":"TWwCRQRZ2ZHMJFn9TzLp7W"},"type":"0"},"txnMetadata":{"seqNo":4,"txnId":"aa5e817d7cc626170eca175822029339a444eb0ee8f0bd20d3b0b76e566fb008"},"ver":"1"}'}
pp.pprint(data)
pp = pprint.PrettyPrinter(indent=4, compact=True)
IndyPrinter(data)('hello')



{   'config': '{"genesis_txn": "/tmp/indy/pool1.txn"}',
    'genesis_txn_data': '{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node1","blskey":"4N8aUNHSgjQVgkpm8nhNEfDf6txHznoYREg9kirmJrkivgL4oSEimFF6nsQ6M41QvhM2Z33nves5vfSn9n1UwNFJBYtWVnHYMATn76vLuL3zU88KyeAYcHfsih3He6UHcXDxcaecHVz6jhCYz1P2UZn2bDVruL5wXpehgBfBaLKm3Ba","blskey_pop":"RahHYiCvoNCtPTrVtP7nMC5eTYrsUA8WjXbdhNc8debh1agE9bGiJxWBXYNFbnJXoXhWFMvyqhqhRoq737YQemH5ik9oL7R4NTTCz2LEZhkgLJzB3QRQqJyBNyv7acbdHrAT8nQ9UkLbaVL9NBpnWXBTw4LEMePaSHEw66RzPNdAX1","client_ip":"127.0.0.1","client_port":9702,"node_ip":"127.0.0.1","node_port":9701,"services":["VALIDATOR"]},"dest":"Gw6pDLhcBcoQesN72qfotTgFa7cbuqZpkX3Xo6pLhPhv"},"metadata":{"from":"Th7MpTaRZVRYnPiabds81Y"},"type":"0"},"txnMetadata":{"seqNo":1,"txnId":"fea82e10e894419fe2bea7d96296a6d46f50f93f9eeda954ec461b2ed2950b62"},"ver":"1"}\n'
                        '{"reqSignature":{},"txn":{"data":{"data":{"alias":"Node2","blskey":"37rAPpXVoxzKhz7d9gkUe52XuXryuLXoM6P6LbWDB7LSbG62Lsb33sfG

In [220]:
data = {'name': 'Acme',
 'wallet_config': '{"id": "acme_wallet"}',
 'wallet_credentials': '{"key": "acme_wallet_key"}',
 'pool': 2,
 'role': 'TRUST_ANCHOR',
 'wallet': 15,
 'did': 'DYHgp6eSHgiuzJEekw44c2',
 'key': '7qPzsoeMbELi5WDEjSDKhVpsfCfijWAVvvmf2SG5tYM4',
 'job_certificate_schema_id': 'HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2',
 'job_certificate_schema': '{"ver":"1.0","id":"HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2","name":"Job-Certificate","version":"0.2","attrNames":["salary","employee_status","first_name","last_name","experience"],"seqNo":106}',
 'job_certificate_cred_def_id': 'DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1',
 'job_certificate_cred_def': '{"ver":"1.0","id":"DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1","schemaId":"106","type":"CL","tag":"TAG1","value":{"primary":{"n":"106359825164629924786501458525694773660587449851015381710218073979061837698988296022625218896266866473744993446831887152296262669332244917382107288259233829827834997517307633232818112411274342932908396539683450590053066940577099098539170086642918565659877840060479220160386673913623656415081463339491557520282268587164256218571644755885676388339380766612502271942964038694921811179026721731107322079717966206156854514846584493762521228339317164056387486421793612636486043530892240900459046266123830655058642474758161933009099336899629352439004056473393907954327647293606065268193743181653871320462412303112921257494329","s":"103838917686892344763162819053104074631315993263293070566220071490778886514082965322872601152292479360976070129942732128965600396127078802596625219081242301740949689005663518270032459854976867681354692959193266998789818672986580166788386847879571581357194379285351221245502024920808916282724477191307662567879051112657359285277469101912763779303375234351584248033049634603485062015534248045582315643679686077209792521903397577534205409708423150745926776302171649882643501993011515470426164131473139691042713260985457374266724589755167262096887611272865827647664784641628786933211108018590246237091176929342983965147437","r":{"salary":"37941139535912486589778852437809463669610657278177764313050693574206734999530746191588054294291782737532938184672916707480000671515820937587336537447452611870931450252822280356722298147661456458995563647698237971714392011794810199941851752192103927384628429162469662400881504512195306164807622058694799628139518785295627480726286718293329333805630298363414661232290108973681809929765945057719091693428762632351769940910498582778887635626765783237822918676446715876886997884667259047618669999652325334104745657663790354901074241409446182998082004521515918873708796816047392382929972389130990040851347557534834315541243","master_secret":"11064733842460954385252835962088991032067346643598800190085726362118265758568527526146479198187004005253236037471264665755569990875949640787326218454340666453701295607412904267578610339869197564157044305844245405226267209721167110165912662270127593424676161762341617940184252638735809096943574315292517305346125794402375732302464173139986665028146136479708642283431848272446986277459670075487968245329616751913804240463663209162738075707844080946978103888713048177136666409533570276978441347425656708577173757871897479302225811972665398920454757110620254975174841388743986035209527010981161186963463024820758796035557","first_name":"94346990387299283804050463071383787112451449807510941499811142275436544887852891801165209943149726971159078010595245046419445134364005932964031497772028669209818583250742130936606543382131062158053655673957925766612972642693500961978527813441504193207305341645331846995470998920411480218595829058204717537732957617595925137510394149885470922245994099599218847280620540503815718799010881645621940333776916322024523734902981171258436939439205562563286001172896142623350086438678182846763558539147242781676070856787448761380608841167696060157958405980570162126612288866463223248965737770803271662862369282938392687023565","experience":"39291248926121066995517204815711687516046609714468940289167209364702270403438929160246000045194910685380976076608545677348182694656120334205017998986820394441993437496734753297990093471150421207398410179059037419409911344567346501586701186433729815214295165612619338631671854104352883600181725249623063680456451827781982388241980664907176531217806307107536103604240692643771730542588060967018191643472384373503486439915441765979162658293037512984745911060803866344855789225500206768316673616232103020084020294109899484823969687524038774745014695789540599327994017032400818708377380094348692537006687323180402348403141","last_name":"91933539312102069056652249674940337512470266662724865122886846477671099198499857763558350196720873136474870320166409680328667403394161730938479389057548427726616732650491926718084138566666167832523544469208606295863620193252795161211578791813199442015952711499796085072785512582515149173937374432983759298581906802853036669645285981363432557695107739211489732205719882470510548141938427354333699027474243260452212434073764896187065167606209149782884100381115053913199418064269056661942821205846599066367217972877156084210422544528463481537748808764365471903809104839978467027972556749095308993966357937459640426786478","employee_status":"72719785260943455525662758244614444612489130311253468975550454369371088980759913555971417306143002992574660308360662087287577712171311254953439668949512654850804260585364490018235367299347739776213221318815598707761363128035572824058100815944367579946441263276333983140139285365199712392895331149782013206366481771687354497455356530923638195708745979082191833161238800860544482683613817381076376033587193250639834708133878263218844907554633547927018293678828856983938779963312248733097215978023259432708727663849027336481651124663225238243020920175766799971952175144167902846002182296537604257955861381807507464159303"},"rctxt":"85942470575577945690771295164600416700212653860492488447980636456032461342178767046869887030322469075425585701136626166582138354180165115283094032893615472974279910708466165684387227234259878655188110153825241838634947932483445728699813304502436429108343220206152905934685079754191941526819485813992807435019429756124075732723628399498527652022739870988813035130868754308344854329185193526562776735708434029351105163564258171079095120477016127562847056869029645829200513690948977757217766021953972290173199811036442505243238274543400305808719777510707714620034672949796152692751498393516041326449046997529751562209730","z":"71349046251484477734364458429601653833470893565388672834849761266221119584243319200576110381346429941481317888106022875441704239138894800152136885215786335382933395168293911176581005324088826612797364854186539544295351596323733028768042686848066722728710976316112035035788286244950397305449602734357151825634851832077391400833048264631641957490384427379476737282236940799315969842556586571943716065174567216570515600641949321903906151973312085485641641684590440512222426930861995417285324509995389372431107872257472432337347905106451090827475379847102686193324767361393563152935662275283712327867604379281191249839993"},"revocation":{"g":"1 0C6E4D9650D2CA2A867A68094989D43B962B62AEEC1694011F2FEE646566676A 1 138C889A9246817100FC8DDBE19D3B433EB2ABB8E02B7FC3ED2ECAD995430F76 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","g_dash":"1 21F6913227FCE1A538A8A3F23A83DAE78FF7CDCCE1FBB408CB487791B2DCA531 1 1DED6139E03815C1FCDE2FB2876D606B4E78AEFBE98FC7643636035C0F1D3EC7 1 23EFC17F9D43618DC3EF43E5585692DE2CC38ADFD4707EE6961590912BC1ED9C 1 02A62D2B61AEB0C6571B4807437708C892E86D496B6318311B60B706CE67E753 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","h":"1 17C4FD9ACA0D875660A4D81EF97A46EB139781922AC13D2F5E8FD9CCE54F97F0 1 179E35CC41D238E254ACD24CC5A11129BEBD5EFD8C60EA0EFFE3D7A296C7787B 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h0":"1 06FA7A387C48D9B0425635235641776A7367ECB59F13FDB9B996CFE0C8589988 1 0651753AB4E38A54D143E1398F4DB6C5758E1BE58466345B519E7B4F06C16396 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h1":"1 07654F2150927D98A69F39DA1834E14009BC76649C72E1AA6456A6FDBA6A90FC 1 0F2C984420F5B22869D185BCA83EA2B1719991DCA2C6D7D1BFCFE8D50C2390A8 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h2":"1 0463B0FBD533F0FFE409BD1E6F1F5D50C8865F86A756016D8DFA22EF4C8B3442 1 1BB3CCA627E08BA32FEAB802B019BAC7F25B106711414CB05A245D7AB41355EE 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","htilde":"1 14090E8DC7959EB7ED8F147DFEC400FEA3E51A108169ADE95D5C1CF834299D6F 1 1FFD5F0390CBBF33AC7B40D4B24A9E940329476BE2AC42C19F269C27A4EB3C2D 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h_cap":"1 1EC927C18F459F3A77E281C221384F26BF9E5DCF19056196F4B9E3A716D345FD 1 122740ECB2B22BDB90EF1C59FF6FBC2D53F214794A844867D42E0D3B261219C3 1 0AB313BD26AC6B914A4F6385806BCBD708EF9C12A3F41EA1C511B89D7EAF2016 1 1E139E0229F196619A23CF0F65DCDC9A3BBC262FACC11A030EB5163D68334E69 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","u":"1 111A188EC1A8F2B101430FE2B52833444858D94204D35ABDB9A59A0A84522C4E 1 0B522CD88C5298104FCDFFAB52903BA5911BB62CCC20729915E6743C35EB5105 1 00B2E690AA9F96F54BE42D6E415F44E836E8888333B0B7B38840AEFE29FED690 1 1C6361C3D09F89EB2C88F3BA42099D81E4D7A3040EAD9572B68AF2E7FA6378C9 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","pk":"1 1CDF50BDADD49E0CBD62F16AB90914D1503A0E54D7CA0B0819A98CFFC5CBF3C0 1 0331CC416089E3411C91516A27567BCAAC131DAF92B7E65281F2358D7E38DE2C 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","y":"1 1A5F7B23714BCB1BD3B7002BFB74E3D5C149FAA9DD5AF5A475750DDC477383C9 1 08C59DCD300DC48451EF0A52EB1FEB20C4D6C2DF307B3BE03F1C623E1EA6BDB9 1 130AC18B389E5FE63E7E011EA9902BB289C1F0645ABE42025C765CC30E941109 1 0A8077643801DC5155EE345E0E61062B456BBB19E43CAD6D25FDA736D1D0C293 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000"}}}',
 'tails_writer_config': '{"base_dir": "/tmp/indy_acme_tails", "uri_pattern": ""}'}

IndyPrinter(data)('hello')

----------------------------------------------------------------------------------------------------
-----------------------------------------------hello------------------------------------------------
<json_keys> : ['name', 'wallet_config', 'wallet_credentials', 'pool', 'role', 'wallet', 'did', 'key', 'job_certificate_schema_id', 'job_certificate_schema', 'job_certificate_cred_def_id', 'job_certificate_cred_def', 'tails_writer_config']
{
    "name": "Acme",
    "wallet_config": {
        "id": "acme_wallet"
    },
    "wallet_credentials": {
        "key": "acme_wallet_key"
    },
    "pool": 2,
    "role": "TRUST_ANCHOR",
    "wallet": 15,
    "did": "DYHgp6eSHgiuzJEekw44c2",
    "key": "7qPzsoeMbELi5WDEjSDKhVpsfCfijWAVvvmf2SG5tYM4",
    "job_certificate_schema_id": "HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2",
    "job_certificate_schema": {
        "ver": "1.0",
        "id": "HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2",
        "name": "Job-Certificate",
        "version": "0.

In [221]:
data = {'name': 'Acme',
 'wallet_config': '{"id": "acme_wallet"}',
 'wallet_credentials': '{"key": "acme_wallet_key"}',
 'pool': 2,
 'role': 'TRUST_ANCHOR',
 'wallet': 15,
 'did': 'DYHgp6eSHgiuzJEekw44c2',
 'key': '7qPzsoeMbELi5WDEjSDKhVpsfCfijWAVvvmf2SG5tYM4',
 'job_certificate_schema_id': 'HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2',
 'job_certificate_schema': '{"ver":"1.0","id":"HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2","name":"Job-Certificate","version":"0.2","attrNames":["salary","employee_status","first_name","last_name","experience"],"seqNo":106}',
 'job_certificate_cred_def_id': 'DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1',
 'job_certificate_cred_def': '{"ver":"1.0","id":"DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1","schemaId":"106","type":"CL","tag":"TAG1","value":{"primary":{"n":"106359825164629924786501458525694773660587449851015381710218073979061837698988296022625218896266866473744993446831887152296262669332244917382107288259233829827834997517307633232818112411274342932908396539683450590053066940577099098539170086642918565659877840060479220160386673913623656415081463339491557520282268587164256218571644755885676388339380766612502271942964038694921811179026721731107322079717966206156854514846584493762521228339317164056387486421793612636486043530892240900459046266123830655058642474758161933009099336899629352439004056473393907954327647293606065268193743181653871320462412303112921257494329","s":"103838917686892344763162819053104074631315993263293070566220071490778886514082965322872601152292479360976070129942732128965600396127078802596625219081242301740949689005663518270032459854976867681354692959193266998789818672986580166788386847879571581357194379285351221245502024920808916282724477191307662567879051112657359285277469101912763779303375234351584248033049634603485062015534248045582315643679686077209792521903397577534205409708423150745926776302171649882643501993011515470426164131473139691042713260985457374266724589755167262096887611272865827647664784641628786933211108018590246237091176929342983965147437","r":{"salary":"37941139535912486589778852437809463669610657278177764313050693574206734999530746191588054294291782737532938184672916707480000671515820937587336537447452611870931450252822280356722298147661456458995563647698237971714392011794810199941851752192103927384628429162469662400881504512195306164807622058694799628139518785295627480726286718293329333805630298363414661232290108973681809929765945057719091693428762632351769940910498582778887635626765783237822918676446715876886997884667259047618669999652325334104745657663790354901074241409446182998082004521515918873708796816047392382929972389130990040851347557534834315541243","master_secret":"11064733842460954385252835962088991032067346643598800190085726362118265758568527526146479198187004005253236037471264665755569990875949640787326218454340666453701295607412904267578610339869197564157044305844245405226267209721167110165912662270127593424676161762341617940184252638735809096943574315292517305346125794402375732302464173139986665028146136479708642283431848272446986277459670075487968245329616751913804240463663209162738075707844080946978103888713048177136666409533570276978441347425656708577173757871897479302225811972665398920454757110620254975174841388743986035209527010981161186963463024820758796035557","first_name":"94346990387299283804050463071383787112451449807510941499811142275436544887852891801165209943149726971159078010595245046419445134364005932964031497772028669209818583250742130936606543382131062158053655673957925766612972642693500961978527813441504193207305341645331846995470998920411480218595829058204717537732957617595925137510394149885470922245994099599218847280620540503815718799010881645621940333776916322024523734902981171258436939439205562563286001172896142623350086438678182846763558539147242781676070856787448761380608841167696060157958405980570162126612288866463223248965737770803271662862369282938392687023565","experience":"39291248926121066995517204815711687516046609714468940289167209364702270403438929160246000045194910685380976076608545677348182694656120334205017998986820394441993437496734753297990093471150421207398410179059037419409911344567346501586701186433729815214295165612619338631671854104352883600181725249623063680456451827781982388241980664907176531217806307107536103604240692643771730542588060967018191643472384373503486439915441765979162658293037512984745911060803866344855789225500206768316673616232103020084020294109899484823969687524038774745014695789540599327994017032400818708377380094348692537006687323180402348403141","last_name":"91933539312102069056652249674940337512470266662724865122886846477671099198499857763558350196720873136474870320166409680328667403394161730938479389057548427726616732650491926718084138566666167832523544469208606295863620193252795161211578791813199442015952711499796085072785512582515149173937374432983759298581906802853036669645285981363432557695107739211489732205719882470510548141938427354333699027474243260452212434073764896187065167606209149782884100381115053913199418064269056661942821205846599066367217972877156084210422544528463481537748808764365471903809104839978467027972556749095308993966357937459640426786478","employee_status":"72719785260943455525662758244614444612489130311253468975550454369371088980759913555971417306143002992574660308360662087287577712171311254953439668949512654850804260585364490018235367299347739776213221318815598707761363128035572824058100815944367579946441263276333983140139285365199712392895331149782013206366481771687354497455356530923638195708745979082191833161238800860544482683613817381076376033587193250639834708133878263218844907554633547927018293678828856983938779963312248733097215978023259432708727663849027336481651124663225238243020920175766799971952175144167902846002182296537604257955861381807507464159303"},"rctxt":"85942470575577945690771295164600416700212653860492488447980636456032461342178767046869887030322469075425585701136626166582138354180165115283094032893615472974279910708466165684387227234259878655188110153825241838634947932483445728699813304502436429108343220206152905934685079754191941526819485813992807435019429756124075732723628399498527652022739870988813035130868754308344854329185193526562776735708434029351105163564258171079095120477016127562847056869029645829200513690948977757217766021953972290173199811036442505243238274543400305808719777510707714620034672949796152692751498393516041326449046997529751562209730","z":"71349046251484477734364458429601653833470893565388672834849761266221119584243319200576110381346429941481317888106022875441704239138894800152136885215786335382933395168293911176581005324088826612797364854186539544295351596323733028768042686848066722728710976316112035035788286244950397305449602734357151825634851832077391400833048264631641957490384427379476737282236940799315969842556586571943716065174567216570515600641949321903906151973312085485641641684590440512222426930861995417285324509995389372431107872257472432337347905106451090827475379847102686193324767361393563152935662275283712327867604379281191249839993"},"revocation":{"g":"1 0C6E4D9650D2CA2A867A68094989D43B962B62AEEC1694011F2FEE646566676A 1 138C889A9246817100FC8DDBE19D3B433EB2ABB8E02B7FC3ED2ECAD995430F76 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","g_dash":"1 21F6913227FCE1A538A8A3F23A83DAE78FF7CDCCE1FBB408CB487791B2DCA531 1 1DED6139E03815C1FCDE2FB2876D606B4E78AEFBE98FC7643636035C0F1D3EC7 1 23EFC17F9D43618DC3EF43E5585692DE2CC38ADFD4707EE6961590912BC1ED9C 1 02A62D2B61AEB0C6571B4807437708C892E86D496B6318311B60B706CE67E753 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","h":"1 17C4FD9ACA0D875660A4D81EF97A46EB139781922AC13D2F5E8FD9CCE54F97F0 1 179E35CC41D238E254ACD24CC5A11129BEBD5EFD8C60EA0EFFE3D7A296C7787B 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h0":"1 06FA7A387C48D9B0425635235641776A7367ECB59F13FDB9B996CFE0C8589988 1 0651753AB4E38A54D143E1398F4DB6C5758E1BE58466345B519E7B4F06C16396 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h1":"1 07654F2150927D98A69F39DA1834E14009BC76649C72E1AA6456A6FDBA6A90FC 1 0F2C984420F5B22869D185BCA83EA2B1719991DCA2C6D7D1BFCFE8D50C2390A8 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h2":"1 0463B0FBD533F0FFE409BD1E6F1F5D50C8865F86A756016D8DFA22EF4C8B3442 1 1BB3CCA627E08BA32FEAB802B019BAC7F25B106711414CB05A245D7AB41355EE 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","htilde":"1 14090E8DC7959EB7ED8F147DFEC400FEA3E51A108169ADE95D5C1CF834299D6F 1 1FFD5F0390CBBF33AC7B40D4B24A9E940329476BE2AC42C19F269C27A4EB3C2D 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","h_cap":"1 1EC927C18F459F3A77E281C221384F26BF9E5DCF19056196F4B9E3A716D345FD 1 122740ECB2B22BDB90EF1C59FF6FBC2D53F214794A844867D42E0D3B261219C3 1 0AB313BD26AC6B914A4F6385806BCBD708EF9C12A3F41EA1C511B89D7EAF2016 1 1E139E0229F196619A23CF0F65DCDC9A3BBC262FACC11A030EB5163D68334E69 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","u":"1 111A188EC1A8F2B101430FE2B52833444858D94204D35ABDB9A59A0A84522C4E 1 0B522CD88C5298104FCDFFAB52903BA5911BB62CCC20729915E6743C35EB5105 1 00B2E690AA9F96F54BE42D6E415F44E836E8888333B0B7B38840AEFE29FED690 1 1C6361C3D09F89EB2C88F3BA42099D81E4D7A3040EAD9572B68AF2E7FA6378C9 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000","pk":"1 1CDF50BDADD49E0CBD62F16AB90914D1503A0E54D7CA0B0819A98CFFC5CBF3C0 1 0331CC416089E3411C91516A27567BCAAC131DAF92B7E65281F2358D7E38DE2C 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8","y":"1 1A5F7B23714BCB1BD3B7002BFB74E3D5C149FAA9DD5AF5A475750DDC477383C9 1 08C59DCD300DC48451EF0A52EB1FEB20C4D6C2DF307B3BE03F1C623E1EA6BDB9 1 130AC18B389E5FE63E7E011EA9902BB289C1F0645ABE42025C765CC30E941109 1 0A8077643801DC5155EE345E0E61062B456BBB19E43CAD6D25FDA736D1D0C293 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000"}}}',
 'tails_writer_config': '{"base_dir": "/tmp/indy_acme_tails", "uri_pattern": ""}',
 'revoc_reg_id': 'DYHgp6eSHgiuzJEekw44c2:4:DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1:CL_ACCUM:TAG1',
 'revoc_reg_def': '{"ver":"1.0","id":"DYHgp6eSHgiuzJEekw44c2:4:DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1:CL_ACCUM:TAG1","revocDefType":"CL_ACCUM","tag":"TAG1","credDefId":"DYHgp6eSHgiuzJEekw44c2:3:CL:106:TAG1","value":{"issuanceType":"ISSUANCE_ON_DEMAND","maxCredNum":5,"publicKeys":{"accumKey":{"z":"1 13373A9FCA6E8E8A12751EE52C184B57C72F76E3AF7E5F6D1920A5327298FE71 1 09EF96C87BF295B578023CC69C5CF9CC7D52E33D1E72F09EC396BE330C1A9A93 1 130DC876C1CD310DF1ABEF61882004A3033A1EACA447C7D94A36E37D816F79B6 1 1FD463913FD441A327102CF6808B2B53E495E3B1599EBE901972184E00D0B7DD 1 0104B475C176D685271A6562E6F3B7BBF88C676C2BE66526F2FA8FFBB3B6F4C3 1 005FC6A72BC2A2EC9D44E4AE4D1205F7AA1BC81D12081B67B5C5902B7FB1952F 1 109E91E52D014EFAC2B7ADFB24F98EDB9E0D21376A9D6C045131883F846AD2AE 1 2004F11E3CCDA5193FF49737BB7FB96C928868EF1DA4B704BBFC7E7302EF93C9 1 2273A3E6A7ED9343C64809E44DF9D7CD660AEAE98A9BCCE53628B150A659BC5C 1 1113CEBBD0B4091A8823B9F873EBBE42F9236954950C3D532E72A0CEA0044FB9 1 1EDD169DC170D699D4585BFCF89FBC846DCD948446383B3633DE4BCA26077B61 1 06B7B2041D2F2B35D5C57722A9BE0207EFFA9B40C7AA36C7D2CC929E2DBDA185"}},"tailsHash":"F85KiXLLyyn2uh5hoiDFseLZR31QTQg6PkocELfwuR4L","tailsLocation":"/tmp/indy_acme_tails/F85KiXLLyyn2uh5hoiDFseLZR31QTQg6PkocELfwuR4L"}}',
 'revoc_reg_entry': '{"ver":"1.0","value":{"accum":"1 0000000000000000000000000000000000000000000000000000000000000000 1 0000000000000000000000000000000000000000000000000000000000000000 2 095E45DDF417D05FB10933FFC63D474548B7FFFF7888802F07FFFFFF7D07A8A8 1 0000000000000000000000000000000000000000000000000000000000000000 1 0000000000000000000000000000000000000000000000000000000000000000 1 0000000000000000000000000000000000000000000000000000000000000000"}}'}

IndyPrinter(data)('hello')

----------------------------------------------------------------------------------------------------
-----------------------------------------------hello------------------------------------------------
<json_keys> : ['name', 'wallet_config', 'wallet_credentials', 'pool', 'role', 'wallet', 'did', 'key', 'job_certificate_schema_id', 'job_certificate_schema', 'job_certificate_cred_def_id', 'job_certificate_cred_def', 'tails_writer_config', 'revoc_reg_id', 'revoc_reg_def', 'revoc_reg_entry']
{
    "name": "Acme",
    "wallet_config": {
        "id": "acme_wallet"
    },
    "wallet_credentials": {
        "key": "acme_wallet_key"
    },
    "pool": 2,
    "role": "TRUST_ANCHOR",
    "wallet": 15,
    "did": "DYHgp6eSHgiuzJEekw44c2",
    "key": "7qPzsoeMbELi5WDEjSDKhVpsfCfijWAVvvmf2SG5tYM4",
    "job_certificate_schema_id": "HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2",
    "job_certificate_schema": {
        "ver": "1.0",
        "id": "HwnVC4oxuamPhWWerqRiao:2:Job-Certificate:0.2",
     